# Train model to drive and takeover obstacles thanks to images and lidar informations

Author : Johan Jublanc
    
Date : 14/11/2019

Description : 

Load the images, lidar information and action to train a model to drive and avoid obstacles.

In [ ]:
import json
import shutil
import random
import pathlib
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import IPython.display as display
from matplotlib.animation import FuncAnimation
from IPython.core.display import display, HTML

import mlflow
import mlflow.tensorflow
import mlflow.keras

import tensorflow as tf

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.losses import MSE, MSLE
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from xebikart.images import transformer as T
import xebikart.dataset as dataset

from sklearn.model_selection import train_test_split

%matplotlib inline

In [ ]:
tf.__version__

In [ ]:
tf.test.is_gpu_available()

Eager Execution allows to evaluate operations immediately without building graphs
note : Only needed when not using TF 2.0

In [ ]:
tf.enable_eager_execution()

In [ ]:
# parameters
# dataset parameters
tubes_root_folder = "file:/workspace/xebikart-ml-tubes"
tubes_folders = ["tub_42_19-11-19"]

test_size=0.2

# training parameters
batch_size = 16
shuffle_size = 200
n_epochs = 50
learning_rate = 1e-4

# Load data

Download tubes from : https://github.com/xebia-france/xebikart-ml-tubes

In [ ]:
raw_tubes_df = dataset.get_tubes_df(tubes_root_folder, tubes_folders, tubes_extension=".tar.gz")
tubes_df = raw_tubes_df.rename(columns={"cam/image_array": "images_path", 
                                        "user/angle": "angles",
                                        "user/throttle": "throttles",
                                        "lidar/distances":"lidar/output"}).reset_index(drop=True)
tubes_df.count()

In [ ]:
tubes_df

# Load data

#### **- Custom lidar data**

In [ ]:
tubes_df["lidar/clipped_distances"] = tubes_df["lidar/output"].apply(lambda distances: np.clip(distances, 0, 600))

#### **- Display some examples**

In [ ]:
random_values = tubes_df.sample(3).reset_index().iterrows()

fig = plt.figure(figsize=(17,10))

for n, sample in random_values:
    
    lidar_360 = sample['clipped_distances']
    random_image_path = sample["images_path"]
    angle = sample["angles"]
    throttle = sample["throttles"]
    image = mpimg.imread(random_image_path)
    
    plt.subplot(2, 3, n+1)
    plt.imshow(image)
    plt.title("ang : {0:.3f}  /  throttle : {0:.3f}".format(angle, throttle))
    
    plt.subplot(2, 3, n+4, projection='polar')
    plt.plot(np.deg2rad(range(360)), lidar_360)

#### **- Display some sample distribution**

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(15,5))
axs[0].hist(tubes_df.angles)
axs[0].set_title('distribution angles')
axs[1].hist(tubes_df.throttles)
axs[1].set_title('distribution throttles')

plt.show()

#### **- Display some gif**

In [ ]:
num_tubes = 300
offset_tubes = 250

rows = tubes_df[offset_tubes:(offset_tubes+num_tubes)].iterrows()

# Create figure
fig = plt.figure(figsize=(17,10))
fig.set_tight_layout(True)

# Axes (img, lidar)
ax_im = fig.add_subplot(1, 2, 1)
ax_lid = fig.add_subplot(1, 2, 2, projection='polar')

def update(df_row):
    index, row = df_row
    
    # Plot
    ax_im.imshow(mpimg.imread(row["images_path"]))
    ax_im.set_xlabel("ang : {0:.3f}  /  throttle : {0:.3f}".format(row["angles"], row["throttles"]))
    ax_lid.clear()
    ax_lid.plot(np.deg2rad(range(360)), row['lidar/clipped_distances'])
    return ax_im, ax_lid

anim = FuncAnimation(fig, update, frames=rows, interval=50)

In [ ]:
#anim.save('lidar.gif', dpi=80, writer='imagemagick')
display(HTML(anim.to_jshtml()))

# Preprocessing Images

**Images will be :**
- Loaded
    - Read images
    - Decode jpeg images into uint8 tensor
- Cropped
    - Crop images on the lower part
- Augmented
    - Brightness : Adjust the brightness of images by a random factor.
    - Saturation : Adjust the saturation of images by a random factor (must be RGB images)
    - Contrast : Adjust the contrast of images by a random factor.
    - Jpeg quality : Randomly changes jpeg encoding quality for inducing jpeg noise
- Normalized
    - Image are converted into Float32 between 0 and 1
- Edged
    - Convert tensor uint8 type into float32 type
    - Convert rgb images to grayscale
    - Reshape into [1, 80, 160, 1] tensor
    - Apply sobel filter (see https://en.wikipedia.org/wiki/Sobel_operator)
    - Reshape into [80, 160, 2] tensor
    - Select image gradient up to 0.3
    - Binarize images by setting elements to 0 or 1

##### **- Display some examples before and after preprocessing**

In [ ]:
crop_fn = T.generate_crop_fn(left_margin=0, width=160, height_margin=40, height=80)
clip_max = 600

def lidar_preprocess(lidar):
    lidar = tf.clip_by_value(lidar, clip_value_min = 0, clip_value_max=clip_max)
    lidar = 1 - lidar/clip_max
    return lidar

def load_augmentation_preprocess(image_path):
    tf_image = T.read_image(image_path)
    tf_image = T.normalize(tf_image)
    tf_image = crop_fn(tf_image)
    tf_image = T.data_augmentation(tf_image)
    tf_image = T.edges(tf_image)
    return tf_image

In [ ]:
random_image_path_lidar_output = tubes_df.sample()[["images_path","lidar/output"]].values[0]
random_image_path = random_image_path_lidar_output[0]

tf_image_original   = T.read_image(random_image_path)
tf_image_cropped    = crop_fn(tf_image_original)
tf_image_augmented  = T.data_augmentation(tf_image_cropped)
tf_image_normalized = T.normalize(tf_image_augmented)
tf_image_edged      = T.edges(tf_image_normalized)

fig, axs = plt.subplots(1, 5, figsize=(15,15), constrained_layout=True)
axs[0].set_title("Original")
axs[0].imshow(tf_image_original)
axs[1].set_title("Cropping")
axs[1].imshow(tf_image_cropped)
axs[2].set_title("Augmented")
axs[2].imshow(tf_image_augmented)
axs[3].set_title("Preprocessed channel 1")
axs[3].imshow(tf_image_edged[:,:,0],cmap='gray')
axs[4].set_title("Preprocessed channel 2")
axs[4].imshow(tf_image_edged[:,:,1],cmap='gray')
plt.show()

# Build a dataset of images

#### **- Split data into test/train datasets**

Note : We only use angle as label

In [ ]:
images_path_and_lidar_outputs = [x for x in zip(tubes_df["images_path"].tolist(), tubes_df["lidar/output"].tolist())]
metas_angle = tubes_df["angles"].tolist()
metas_throttle = tubes_df["throttles"].tolist()

In [ ]:
train_images_path_lidar_outputs, test_images_path_lidar_outputs, train_metas, test_metas = train_test_split(images_path_and_lidar_outputs, metas_angle, test_size=test_size)
print('Train set :', len(train_images_path_lidar_outputs), 'images')
print('Test set :', len(test_images_path_lidar_outputs), 'images')

#### **- Create tensor for train and test datasets**

In [ ]:
def input_fn(filepath, lidar, label, BATCH_SIZE = 32, SHUFFLE_SIZE = 200, NUM_EPOCHS = 50, NUM_REPEAT=200):
    ds_x = tf.data.Dataset.from_tensor_slices(filepath)
    ds_x = ds_x.map(load_augmentation_preprocess)
    ds_l = tf.data.Dataset.from_tensor_slices(lidar)
    ds_l = ds_l.map(lidar_preprocess)
    ds_y = tf.data.Dataset.from_tensor_slices(label)
    ds_xl_y = tf.data.Dataset.zip(((ds_x, ds_l), ds_y)).shuffle(SHUFFLE_SIZE).repeat(NUM_REPEAT).batch(BATCH_SIZE).prefetch(1)
    
    return ds_xl_y

In [ ]:
train_images_path = [x[0] for x in train_images_path_lidar_outputs]
train_lidar_outputs = [x[1] for x in train_images_path_lidar_outputs]
test_images_path = [x[0] for x in test_images_path_lidar_outputs]
test_lidar_outputs = [x[1] for x in test_images_path_lidar_outputs]

In [ ]:
print(len(train_lidar_outputs)==len(train_images_path))
print(len(test_lidar_outputs)==len(test_images_path))

In [ ]:
ds_train = input_fn(train_images_path, train_lidar_outputs, train_metas)
ds_test = input_fn(test_images_path, train_lidar_outputs, test_metas)

In [ ]:
ds_train.output_shapes

# Build Model

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras import Input

In [ ]:
main_input = Input(shape = ds_train.output_shapes[0][0][1:4], name='main_input')
lidar_input = Input(shape = (ds_train.output_shapes[0][1][1],), name='lidar_input')

In [ ]:
##########################
## convolutional layers ##
##########################

x = tf.keras.layers.Conv2D(filters = 16,
                           kernel_size = (5,5), 
                           activation='relu', 
                           kernel_regularizer=l1_l2(l1=0.1, l2=0.01))(main_input)
x = tf.keras.layers.MaxPooling2D((3,3))(x)

x = tf.keras.layers.Conv2D(filters = 32,
                           kernel_size = (3,3), 
                           activation='relu', 
                           kernel_regularizer=l1_l2(l1=0.1, l2=0.01))(x)
x = tf.keras.layers.MaxPooling2D((3,3))(x)

x = tf.keras.layers.Conv2D(filters = 64,
                           kernel_size = (3,3), 
                           activation='relu', 
                           kernel_regularizer=l1_l2(l1=0.1, l2=0.01))(x)
# flatten
x = tf.keras.layers.Flatten()(x)
# add lidar inputs


y = tf.keras.layers.Dense(360, activation ='relu')(lidar_input)
y = tf.keras.layers.Dropout(0.6)(y)
y = tf.keras.layers.Dense(180, activation ='relu')(y)
y = tf.keras.layers.Dropout(0.4)(y)
y = tf.keras.layers.Dense(36, activation ='sigmoid')(y)
y = tf.keras.layers.Dropout(0.4)(y)
x = tf.keras.layers.concatenate([x, y])

##################
## dense layers ##
##################

# x = tf.keras.layers.Dense(512, activation='relu')(x)
# x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.6)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.4)(x)
output = tf.keras.layers.Dense(1, activation='linear')(x)

In [ ]:
model = Model(inputs=[main_input, lidar_input], outputs = output)

In [ ]:
model.compile(optimizer=Adam(learning_rate=learning_rate, decay=1e-6), loss="mae")
model.summary()

**- How to choose and interpret loss :**

Mean Absolute Error (MAE) is the absolute value of the substraction of predicted from actual value
Let's see an example : 
Actual angle = 0.21


In [ ]:
mlflow.set_experiment("drive_auto_image_and_lidar")

with mlflow.start_run():
    mlflow.log_params({
        "images": str(tubes_folders),
        "nb_images": len(train_images_path),
        "epochs": n_epochs,
        "batch_size": batch_size,
        "learning_rate": learning_rate
    })
    mlflow.tensorflow.autolog()
    history = model.fit(x=ds_train,
                    steps_per_epoch=len(train_metas)//batch_size,
                    epochs=n_epochs,
                    verbose=1,
                    validation_data=ds_test,
                    validation_steps=len(test_metas)//batch_size)

# Plot loss

In [ ]:
def plot_results(history):
    hist_df = pd.DataFrame(history.history)
    hist_df.columns=['loss','val_loss']
    hist_df.index = np.arange(1, len(hist_df)+1)
    fig = plt.figure(figsize=(10,5))
    plt.plot(hist_df.val_loss, lw=3, label='Validation Loss')
    plt.plot(hist_df.loss, lw=3, label='Training Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.grid()
    plt.legend(loc=0)
    plt.ylim(0,20)
    
    plt.show()

In [ ]:
plot_results(history)

# Evaluation

**- steps:** Total number of steps (batches of samples) before declaring the prediction round finished. Ignored with the default value of None. If x is a tf.data dataset or a dataset iterator, and steps is None, predict will run until the input dataset is exhausted.

In [ ]:
train_angles = model.predict(ds_test, steps=len(test_metas)//batch_size)

In [ ]:
df = pd.DataFrame(train_angles, columns = ['angles'])
df.angles.hist(bins=5)

In [ ]:
df.angles.plot.kde()